In [20]:
import torch
import torch.nn as nn
import torchvision
from torchvision import transforms, datasets, models
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import copy
import time
import os

In [22]:
import utils
from models import ResNet

In [23]:
data_loaders, test_loader = utils.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [24]:
classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [25]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#net = ResNet.ResNet50()
net = ResNet.ResNet18()
net = net.to(device)

print(torch.cuda.current_device())
print(torch.cuda.device(0))
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

0
1
Tesla K80


In [26]:
torch.cuda.is_available()

if torch.cuda.is_available():
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True
    criterion = nn.CrossEntropyLoss().cuda()
else:
    criterion = nn.CrossEntropyLoss()

In [39]:
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4, nesterov=True)
#scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, 'min')
#scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, 32)
scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[81, 122], gamma=0.1)

Size of CIFAR-10

In [35]:
dataset_size = {'train': 4000,'val': 1000,'test': 1000}

Load Trained Model

In [36]:
SAVE_PATH = './trained-models/sgd-net.pth'

#utils.load_checkpoint(net, optimizer, scheduler, SAVE_PATH)

In [37]:
def train_model(model, criterion, optimizer, scheduler, num_epochs):
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_accuracy = 0.0
    
    for epoch in range(num_epochs):
        
        print(str(epoch) + "/" + str(num_epochs))
        
        if type(scheduler) is torch.optim.lr_scheduler.MultiStepLR:
            scheduler.step()
        
        for phase in ['train', 'val']:
            
            print(phase)
            
            if phase == 'train':
                model.train()
                
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0
            total = 0
            
            start = time.time()
            
            for index, (inputs, targets) in enumerate(data_loaders[phase]):
                
                inputs = inputs.to(device)
                targets = targets.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = outputs.max(1)
                    loss = criterion(outputs, targets)
                    
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                        
                running_loss += loss.item()
                running_corrects += preds.eq(targets).sum().item()
                total += targets.size(0)
                
            epoch_loss = running_loss / total
            epoch_acc = running_corrects / total
                
            print('Loss: ' + str(epoch_loss) + ", Epoch Accuracy: " + str(epoch_acc))
            
            print('Time: ' + str((time.time() - start) / 60))
            
            if phase == 'val' and type(scheduler) is torch.optim.lr_scheduler.ReduceLROnPlateau:
                scheduler.step(epoch_loss)
            
            if phase == 'val' and epoch_acc > best_accuracy:
                best_accuracy = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
                if not os.path.isdir('trained-models'):
                    os.mkdir('trained-models')
                
                state = {
                    
                    'epoch': epoch,
                    'state_dict': model.state_dict(),
                    'optimizer': optimizer.state_dict(),
                    'scheduler': scheduler.state_dict()
                    
                }

                if os.path.exists(SAVE_PATH):
                    os.remove(SAVE_PATH)
                
                torch.save(state, SAVE_PATH)
    
    print('Best Accuracy: ' + best_accuracy)
    
    model.load_state_dict(best_model_wts)
    return model

In [38]:
epochs = 200

net = train_model(net, criterion, optimizer, scheduler, epochs)

0/200
train


Process Process-201:
Process Process-204:
Process Process-202:
Process Process-203:
Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ubuntu/src/anaconda3/envs/f

KeyboardInterrupt: 

  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/ubuntu/src/anaconda3/envs/fastai/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/